In [12]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pandas as pd
import cv2
import glob
import xml.etree.ElementTree as ET

import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV3Large
from sklearn.model_selection import train_test_split

import PIL
from PIL import Image 
from PIL.ImageDraw import Draw

2.19.0


In [25]:
def xml_to_csv(path):
    """
    Convert XML files to CSV
    Args:
        path: Path to the folder containing XML files
        skipNegatives: If True, skip images with no burger annotations
    Returns:
        Pandas DataFrame with columns ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    """
    xml_list = []
    
    # Get all XML files in the directory
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # Get image size
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)
        filename = root.find('filename').text

        # Check if there are any burger objects
        objects = root.findall('object')
        
        # Function to safely check if an object is a burger
        def is_burger(obj):
            name_tag = obj.find('name')
            if name_tag is not None and name_tag.text is not None:
                # Remove any whitespace and check if it's 'burger'
                return name_tag.text.strip() == 'burger'
            return False
        
        has_burger = any(is_burger(obj) for obj in objects) if objects else False
        
        if not has_burger:
            xml_list.append({
                'filename': filename,
                'width': width,
                'height': height,
                'class': 0,
                'xmin': 0,
                'ymin': 0,
                'xmax': 0,
                'ymax': 0
            })
        else:
            for obj in objects:
                if obj.find('name').text.strip() == 'burger':
                    bbox = obj.find('bndbox')
                    xml_list.append({
                        'filename': filename,
                        'width': width,
                        'height': height,
                        'class': 1,  # BURGER
                        'xmin': int(bbox.find('xmin').text),
                        'ymin': int(bbox.find('ymin').text),
                        'xmax': int(bbox.find('xmax').text),
                        'ymax': int(bbox.find('ymax').text)
                    })
    
    # Convert to DataFrame
    df = pd.DataFrame(xml_list)
    columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    df = df[columns]
    return df

In [ ]:
df = xml_to_csv('Images')
# Print some statistics
print(f"\nTotal annotations found: {len(df)}")
print(f"Number of burger annotations (class 1): {len(df[df['class'] == 1])}")
print(f"Number of non-burger images (class 0): {len(df[df['class'] == 0])}")

# Save to CSV
output_file = 'Dataset/annotations.csv'


Total annotations found: 1939
Number of burger annotations (class 1): 1483
Number of non-burger images (class 0): 456


In [28]:
df.to_csv(output_file, index=False)
print(f"\nSaved annotations to {output_file}")
print("\nFirst few rows of the CSV:")
print(df.head())


Saved annotations to annotations.csv

First few rows of the CSV:
                                            filename  width  height  class  \
0  1-copy_png.rf.75b0339aa72da0f13d9b38adeea13dbe...    640     360      1   
1    100_png.rf.04808b767e584757447ffa48a789af40.jpg    640     360      1   
2    100_png.rf.11490e40b9e259fabbdb0cedefd2fa65.jpg    640     360      1   
3    100_png.rf.826904cd5effc985fc597d0cd7782e49.jpg    640     360      1   
4    100_png.rf.870050c76f721b9092a0d130584d4337.jpg    640     360      1   

   xmin  ymin  xmax  ymax  
0   170    43   397   295  
1   224   174   452   361  
2   204    81   379   289  
3   189    87   415   324  
4   224    66   421   281  


In [34]:
def split_dataset(csv_path, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=202):
    """
    Split the dataset into train, validation and test sets while maintaining class balance
    
    Args:
        csv_path: Path to the annotations CSV file
        train_ratio: Ratio of training data (default: 0.7)
        val_ratio: Ratio of validation data (default: 0.15)
        test_ratio: Ratio of test data (default: 0.15)
        random_state: Random seed for reproducibility
    """
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Create separate dataframes for positive and negative samples
    negative_samples = df[df['class'] == 0]
    positive_samples = df[df['class'] == 1]
    
    print(f"Total samples: {len(df)}")
    print(f"Positive samples (class 1): {len(positive_samples)}")
    print(f"Negative samples (class 0): {len(negative_samples)}")
    
    # First split: separate test set
    neg_train_val, neg_test = train_test_split(
        negative_samples, 
        test_size=test_ratio, 
        random_state=random_state
    )
    
    pos_train_val, pos_test = train_test_split(
        positive_samples, 
        test_size=test_ratio, 
        random_state=random_state
    )
    
    # Second split: separate train and validation from the remaining data
    val_ratio_adjusted = val_ratio / (train_ratio + val_ratio)
    
    neg_train, neg_val = train_test_split(
        neg_train_val, 
        test_size=val_ratio_adjusted, 
        random_state=random_state
    )
    
    pos_train, pos_val = train_test_split(
        pos_train_val, 
        test_size=val_ratio_adjusted, 
        random_state=random_state
    )
    
    # Combine positive and negative samples for each set
    train_df = pd.concat([pos_train, neg_train])
    val_df = pd.concat([pos_val, neg_val])
    test_df = pd.concat([pos_test, neg_test])
    
    # Shuffle each dataset
    train_df = train_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    val_df = val_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    test_df = test_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    # Print statistics
    print("\nDataset split statistics:")
    print(f"Training set: {len(train_df)} samples")
    print(f"  - Positive (class 1): {len(train_df[train_df['class'] == 1])}")
    print(f"  - Negative (class 0): {len(train_df[train_df['class'] == 0])}")
    
    print(f"\nValidation set: {len(val_df)} samples")
    print(f"  - Positive (class 1): {len(val_df[val_df['class'] == 1])}")
    print(f"  - Negative (class 0): {len(val_df[val_df['class'] == 0])}")
    
    print(f"\nTest set: {len(test_df)} samples")
    print(f"  - Positive (class 1): {len(test_df[test_df['class'] == 1])}")
    print(f"  - Negative (class 0): {len(test_df[test_df['class'] == 0])}")
    
    # Save the splits to CSV files
    train_df.to_csv('train.csv', index=False)
    val_df.to_csv('val.csv', index=False)
    test_df.to_csv('test.csv', index=False)
    
    return train_df, val_df, test_df

In [32]:
train_df, val_df, test_df = split_dataset('Dataset/annotations.csv')

Total samples: 1939
Positive samples (class 1): 1483
Negative samples (class 0): 456

Dataset split statistics:
Training set: 1355 samples
  - Positive (class 1): 1037
  - Negative (class 0): 318

Validation set: 292 samples
  - Positive (class 1): 223
  - Negative (class 0): 69

Test set: 292 samples
  - Positive (class 1): 223
  - Negative (class 0): 69

Saved datasets to:
- train.csv
- val.csv
- test.csv


In [2]:
TRAIN_CSV_FILE = 'Dataset/train.csv'
VAL_CSV_FILE = 'Dataset/val.csv'
TEST_CSV_FILE = 'Dataset/test.csv'
IMAGE_DIR = 'Images'

def prepare_dataset(csv_file, image_dir):
    """
    Prepare images, targets and labels from a CSV file
    
    Args:
        csv_file: Path to CSV file containing annotations
        image_dir: Directory containing the images
        
    Returns:
        images: numpy array of images
        targets: dictionary containing class and bbox outputs
        labels: numpy array of class labels
    """
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    images = []
    bbox_targets = []
    class_labels = []
    
    for index, row in df.iterrows():
        # Extract data from row
        filename = row['filename']
        width = row['width']
        height = row['height']
        class_label = row['class']  # Assuming this is 0 or 1 for binary classification
        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']
        
        # Load and preprocess image
        image_path = os.path.join(image_dir, filename)
        img = keras.preprocessing.image.load_img(image_path, target_size=(360, 640))  # Fixed size for all images
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = img_array / 255.0  # Normalize pixel values
        
        # Normalize bounding box coordinates
        bbox = [
            round(xmin/width, 4),
            round(ymin/height, 4),
            round(xmax/width, 4),
            round(ymax/height, 4)
        ]
        
        images.append(img_array)
        bbox_targets.append(bbox)
        class_labels.append(class_label)
    
    # Convert lists to numpy arrays
    images = np.array(images, dtype=np.float32)
    bbox_targets = np.array(bbox_targets, dtype=np.float32)
    class_labels = np.array(class_labels, dtype=np.float32)
    
    # Create targets dictionary for multi-output model
    targets = {
        'class_output': class_labels,
        'bbox_output': bbox_targets
    }
    
    return images, targets

# Prepare train, validation and test sets
print("Preparing training data...")
train_images, train_targets = prepare_dataset(TRAIN_CSV_FILE, IMAGE_DIR)

print("Preparing validation data...")
val_images, val_targets = prepare_dataset(VAL_CSV_FILE, IMAGE_DIR)

print("Preparing test data...")
test_images, test_targets = prepare_dataset(TEST_CSV_FILE, IMAGE_DIR)

# Print shapes to verify
print("\nDataset shapes:")
print(f"Training images: {train_images.shape}")
print(f"Training class labels: {train_targets['class_output'].shape}")
print(f"Training bbox targets: {train_targets['bbox_output'].shape}")
print(f"\nValidation images: {val_images.shape}")
print(f"Validation class labels: {val_targets['class_output'].shape}")
print(f"Validation bbox targets: {val_targets['bbox_output'].shape}")
print(f"\nTest images: {test_images.shape}")
print(f"Test class labels: {test_targets['class_output'].shape}")
print(f"Test bbox targets: {test_targets['bbox_output'].shape}")

Preparing training data...
Preparing validation data...
Preparing test data...

Dataset shapes:
Training images: (1355, 360, 640, 3)
Training class labels: (1355,)
Training bbox targets: (1355, 4)

Validation images: (292, 360, 640, 3)
Validation class labels: (292,)
Validation bbox targets: (292, 4)

Test images: (292, 360, 640, 3)
Test class labels: (292,)
Test bbox targets: (292, 4)


In [8]:
def create_model():
    base_model = MobileNetV3Large(
        input_shape=(360,640,3),
        include_top=False, 
        weights='imagenet'
    )
    base_model.trainable = True

    x = base_model.output

    x = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    ########## block 1 ##########
    x = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x_shortcut = x

    for i in range(2):
        x = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Add()([x_shortcut, x])
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x_shortcut = x


    ########## block 2 ##########
    x = tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x_shortcut = x

    for i in range(2):
        x = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Add()([x_shortcut, x])
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x_shortcut = x

    ########## block 3 ##########
    x = tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x_shortcut = x

    for i in range(3):
        x = tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Add()([x_shortcut, x])
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x_shortcut = x

        
    ########## block 4 ##########
    x = tf.keras.layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x_shortcut = x

    for i in range(5):
        x = tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Add()([x_shortcut, x])
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x_shortcut = x

    ########## block 5 ##########
    x = tf.keras.layers.Conv2D(1024, (3, 3), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x_shortcut = x

    for i in range(8):
        x = tf.keras.layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Conv2D(1024, (3, 3), strides=(1, 1), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x = tf.keras.layers.Add()([x_shortcut, x])
        x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

        x_shortcut = x

    ########## output layers ##########
    x = tf.keras.layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x = tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    x = tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(negative_slope=0.1)(x)

    # Classification head
    class_branch = tf.keras.layers.GlobalAveragePooling2D()(x)
    class_branch = tf.keras.layers.Dense(64, activation='relu')(class_branch)
    class_output = tf.keras.layers.Dense(1, activation='sigmoid', name='class_output')(class_branch)

    # Bounding box head
    bbox_branch = tf.keras.layers.GlobalAveragePooling2D()(x)
    bbox_branch = tf.keras.layers.Dense(64, activation='relu')(bbox_branch)
    bbox_output = tf.keras.layers.Dense(4, activation='sigmoid', name='bbox_output')(bbox_branch)

    model = tf.keras.Model(
        inputs=base_model.input, 
        outputs={
            'class_output': class_output,
            'bbox_output': bbox_output
        }
    )
    return model

model = create_model()

In [9]:
def custom_loss(y_true, y_pred):
    binary_crossentropy = prob_loss = tf.keras.losses.BinaryCrossentropy(
        reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
    )
    
    prob_loss = binary_crossentropy(
        tf.concat([y_true[:,:,:,0], y_true[:,:,:,5]], axis=0), 
        tf.concat([y_pred[:,:,:,0], y_pred[:,:,:,5]], axis=0)
    )
    
    xy_loss = tf.keras.losses.MSE(
        tf.concat([y_true[:,:,:,1:3], y_true[:,:,:,6:8]], axis=0), 
        tf.concat([y_pred[:,:,:,1:3], y_pred[:,:,:,6:8]], axis=0)
    )
    
    wh_loss = tf.keras.losses.MSE(
        tf.concat([y_true[:,:,:,3:5], y_true[:,:,:,8:10]], axis=0), 
        tf.concat([y_pred[:,:,:,3:5], y_pred[:,:,:,8:10]], axis=0)
    )
    
    bboxes_mask = get_mask(y_true)
    
    xy_loss = xy_loss * bboxes_mask
    wh_loss = wh_loss * bboxes_mask
    
    return prob_loss + xy_loss + wh_loss

def get_mask(y_true):
    anchor_one_mask = tf.where(
        y_true[:,:,:,0] == 0, 
        0.5, 
        5.0
    )
    
    anchor_two_mask = tf.where(
        y_true[:,:,:,5] == 0, 
        0.5, 
        5.0
    )
    
    bboxes_mask = tf.concat(
        [anchor_one_mask,anchor_two_mask],
        axis=0
    )
    
    return bboxes_mask

In [10]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss={
        'class_output': 'binary_crossentropy',
        'bbox_output': 'mse'
    },
    metrics={
        'class_output': 'accuracy',
        'bbox_output': 'mse'
    }
)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 360, 640,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 360, 640,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 180, 320,  │        432 │ rescaling_2[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 180, 320,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_40       │ (None, 180, 320,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 180, 320,  │        144 │ activation_40[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 180, 320,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_38 (ReLU)     │ (None, 180, 320,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 180, 320,  │        256 │ re_lu_38[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 180, 320,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 180, 320,  │          0 │ activation_40[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 180, 320,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 180, 320,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_39 (ReLU)     │ (None, 180, 320,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 181, 321,  │          0 │ re_lu_39[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 90, 160,   │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 90, 160,   │        256 │ expanded_conv_1_

 Total params: 105,304,037 (401.70 MB)

 Trainable params: 105,238,101 (401.45 MB)

 Non-trainable params: 65,936 (257.56 KB)

In [13]:
callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            save_best_only=True,
            monitor='val_loss'
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            restore_best_weights=True
        )
    ]

history = model.fit(
        train_images,
        train_targets,
        validation_data=(val_images, val_targets),
        epochs=64,
        batch_size=32,
        callbacks=callbacks
    )

Epoch 1/64


InvalidArgumentError: Graph execution error:

Detected at node adam/Sub_1487 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Admin\Desktop\IDC-Grp5\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\Admin\Desktop\IDC-Grp5\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Admin\Desktop\IDC-Grp5\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\Admin\Desktop\IDC-Grp5\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 645, in run_forever

  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1999, in _run_once

  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\asyncio\events.p

In [8]:
cam = cv2.VideoCapture(1) 
cam.set(cv2.CAP_PROP_FPS, 120)

while True:
    ret, image = cam.read()
    cv2.imshow('Camera Feed', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


1   HIToolbox                           0x000000018ecf45c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x000000018ecbe478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x000000018a1d1214 _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x000000018a1cfe10 CGSSnarfAndDispatchDatagrams + 808
5   SkyLight                            0x000000018a4f7a9c SLSGetNextEventRecordInternal + 344
6   SkyLight                            0x000000018a2f8fb0 SLEventCreateNextEvent + 16
7   HIToolbox                           0x000000018ec8cb58 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 60
8   HIToolbox                           0x000000018ec8cae0 _ZL14MessageHandlerP12__CFMachPortPvlS1_ + 60
9   CoreFoundation                      0x00000001855d1410 __CFMachPortPerform + 260
10  CoreFoundation                      0x00000001855a1f98 __CFRUNL

KeyboardInterrupt: 

: 